In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import warnings
import copy
warnings.filterwarnings('ignore')

In [2]:
def number(mas):
    """
    Функция вычисляет сумму по строке вида 'a1 a2...an x', 
    где ai - числа, получающиеся при записи числа по 3 цифры через пробел, а x - единица измерения
    """
    s = ''
    for item in mas:
        s += item
    return(int(s))

In [3]:
def MotoParser(soup: BeautifulSoup, brand: str):
    """Данная функция по html коду сайта auto.ru находит нужную информацию по мотоциклам и записывает её в Motocycles.csv"""


    mess = [item.text for item in soup.find_all('div', class_ = 'ListingItemTechSummaryDesktop__cell')]
    # Получаем данные по характеристикам двигателя, коробке, приводе и цвете мотоцикла

    i = 1
    j = 0
    data = dict()
    x = mess[j].split('/')
    if len(x) == 0:
        x.extend(['', ''])
    elif len(x) == 1:
        if 'см³' not in x[0]:
            x.insert(0, '')
            x[1] = number(x[1].split()[:-1])
        else:
            x.insert(1, '')
            x[0] = number(x[0].split()[:-1])
    elif len(x) >= 2:
        if 'см³' not in x[0]:
            x.insert(0, '')
            x[1] = number(x[1].split()[:-1])
        elif 'л.с.' not in x[1]:
            x.insert(1, '')
            x[0] = number(x[0].split()[:-1])
        elif 'см³' in x[0] and 'л.с.' in x[1]:
            x[0] = number(x[0].split()[:-1])
            x[1] = number(x[1].split()[:-1])
    x = x[0:2]
    data[j] = []
    data[j].extend(x)
    # Первый элемент был положен в словарь, поскольку дальше используем вид данных при парсинге:
    # Если перед элементом, начинающимся на цифру, идёт элемент, начинающися на букву, то это означает новый объект

    while i != len(mess): # Обрабатываем данные, приводя их к виду словаря
        if mess[i][0].isnumeric() and not mess[i-1][0].isnumeric():
            x = mess[i].split('/')
            if len(x) == 0:
                x.extend(['', ''])
            elif len(x) == 1:
                if 'см³' not in x[0]:
                    x.insert(0, '')
                    x[1] = number(x[1].split()[:-1])
                else:
                    x.insert(1, '')
                    x[0] = number(x[0].split()[:-1])
            elif len(x) >= 2:
                if 'см³' not in x[0]:
                    x.insert(0, '')
                    x[1] = number(x[1].split()[:-1])
                elif 'л.с.' not in x[1]:
                    x.insert(1, '')
                    x[0] = number(x[0].split()[:-1])
                elif 'см³' in x[0] and 'л.с.' in x[1]:
                    x[0] = number(x[0].split()[:-1])
                    x[1] = number(x[1].split()[:-1])
            x = x[0:2]
            j += 1
            data[j] = []
            data[j].extend(x)

        else:
            if mess[i][0].isnumeric():
                data[j].append(mess[i].replace('\u2009', ''))
            else:
                data[j].append(mess[i])
        i += 1

    transmission = ['Робот', 'передач', 'АКПП', 'Вариатор', 'прямых', 'автомат']
    for i in range(len(data)): # Опытным путём обнаружили, что в основном каждый объект обладает шестью
        if len(data[i]) != 6:  # характеристиками по выбранному участку html-кода => дополним пустотами пропуски
            if 'цилиндр' not in data[i][2]:
                data[i].insert(2, '')
            if sum([item in data[i][3] for item in transmission]) == 0:
                data[i].insert(3, '')
            if data[i][4] not in ['кардан', 'цепь', 'ремень']:
                data[i].insert(4, '')
            if len(data[i]) != 6:
                data[i].append('')

    Name = [item.text for item in soup.find_all('a', class_ = 'Link ListingItemTitle__link')]
    Range = [number(item.text.split()[:-1]) for item in soup.find_all('div', class_ = 'ListingItem__kmAge')]
    Price = [number(item.text.split()[:-1]) for item in soup.find_all('div', class_ = 'ListingItemPrice__content')]
    Location = [item.text for item in soup.find_all('span', class_ = 'MetroListPlace__regionName MetroListPlace_nbsp')]
    Year = [int(item.text) for item in soup.find_all('div', class_ = 'ListingItem__year')]
    # По другим участкам html-кода находим модель, пробег и цену мотоциклов, а также город продажи

    k = len(data)
    dct = copy.deepcopy(data)
    if len(Name) == k and len(Location) == k and len(Range) == k and len(Price) == k:
        for i in range(k):
            dct[i].insert(0, brand)
            dct[i].insert(1, Name[i])
            dct[i].insert(2, Year[i])
            dct[i].insert(3, Location[i])
            dct[i].insert(4, Range[i])
            dct[i].append(Price[i])
    else:
        print('Что-то не так')
    # Добавим полученные характеристики в data в виде словаря, если количество строк, 
    # т.е. предполагаемых объектов, одинаково для каждой характеристики

    df = pd.DataFrame.from_dict(dct, orient = 'index',
                           columns = ['Марка', 'Модель', 'Год', 'Город', 'Пробег(км)', 'Объем двигателя (см³)',
                                      'Мощность двигателя (л.с.)', 'Цилиндров', 'Коробка', 'Привод', 'Цвет', 'Цена(руб)'])
    # Здесь оставлены названия стоблцов, для того, чтобы возникала ошибка в случае неправильной обработки информации

    df.to_csv('Motocycles', mode = 'a', index = False, header = False)
    # Приведём всё к датафрейму, а затем добавим в файл Motocycles.csv, при этом опустим индекс и названия столбцов

In [4]:
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
driver = webdriver.Chrome(options=chrome_options) # Откроем Chrome браузер, отключив разрешения
driver.get('https://auto.ru/rossiya/motorcycle/used/') # Откроем ссылку с перемешанными мотоциклами 

**На данном этапе лучше ввести данные для входа в свой аккаунт, потому что после регистрации как-будто не так часто всплывает реклама, капча, или та же просьба зарегистрироваться**

In [5]:
button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'Все марки — 210')]")))
button.click() #Нажмем на кнопку, которая раскроет все марки мотоциклов + ссылки на них при нажатии

page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
brands = [[item.text, item.get('href')] for item in soup.find_all('a', class_ = 'Link ListingPopularMMM__itemName')]
brands.remove(['Другие', 'https://auto.ru/rossiya/motorcycle/drugie/used/'])
# Найдем все марки и ссылки на них
# Cсылки нужны, потому что автоматически их предугадать не получится из-за перевода русского в английский в ссылке
# Удалим элемент, где на месте марки фраза "Другие", потому что мотоциклы в данном блоке будет сложно оценить
# Всё из-за того, что они кастомные. Также сама марка "другие" не несёт для нас никакой ценности + их немного => не берём данные

In [9]:
line = 'Button Button_color_whiteHoverBlue Button_size_s Button_type_link Button_width_default ListingPagination__page'
df = pd.DataFrame(columns = ['Марка', 'Модель', 'Год', 'Город', 'Пробег(км)', 'Объем двигателя (см³)',
                                      'Мощность двигателя (л.с.)', 'Цилиндров', 'Коробка', 'Привод', 'Цвет', 'Цена(руб)'])
df.to_csv('Motocycles', mode = 'a', index = False)
# Изначально введем оглавления нашего файла
# Далее добавлять данные будем без объявления имени столбцов (чтобы не было дубликатов)

for brand, link in tqdm(brands):
    try:
        driver.get(f"{link}")
        page = driver.page_source
        soup = BeautifulSoup(page, 'html.parser')
        try:
            num_of_pages = int([item.text for item in soup.find_all('a', class_ = line)][-1])
        except:
            num_of_pages = 1
            # num_of_pages - количество страниц для определнной марки мотоцикла
            # При отсутствии определенной части кода делаем вывод, что страница одна
        MotoParser(soup, brand)
        time.sleep(1.5)
        for i in range(2, num_of_pages): # Пробегаемся по всем страницам определенной марки и добаляем инфу в Motocycles.csv
            try:
                driver.get(f"{link}?page={i}")
                page = driver.page_source
                soup = BeautifulSoup(page, 'html.parser')
                MotoParser(soup, brand)
                time.sleep(1.5)
            except:
                # В случае ошибки 1) доступа к странице или 2) при применении функции,
                # выводим марку и номер страницы, где она произошла
                print(f"{brand}: {i}")
                break
    except:
        # В случае ошибки 1) доступа к странице или 2) при применении функции для единственной страницы определенной марки,
        # выводим марку, где она произошла
        print(brand)
        continue

  0%|          | 0/209 [00:00<?, ?it/s]

In [10]:
driver.close() # Закрываем браузер